In [14]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import date
import json
import tensorflow as tf

In [15]:
from keras.models import model_from_json

In [2]:
url='https://pkgstore.datahub.io/sports-data/english-premier-league/season-1819_json/data/121aec954d44d69659e8da82196f0997/season-1819_json.json'
r = requests.get(url)
r_json=r.json()

In [3]:
json_values=json.dumps(r_json)

In [4]:
data=pd.read_json(json_values)

In [5]:
data.to_csv('clean_data/latest_szn_values.csv')

In [6]:
df=pd.read_csv(r'clean_data/latest_szn_values.csv')

In [7]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
old_data=pd.read_csv('clean_data/clean_data.csv')

In [9]:
h=old_data[old_data['HomeTeam']=='Arsenal']

In [10]:
from DataCleaning import DataCleaner

In [11]:
data2=DataCleaner('clean_data/latest_szn_values.csv','clean_data/szn_2017_2018.csv')

/home/addybrown/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


   Unnamed: 0  AC  AF  AR  AS  AST  AY        AwayTeam       Date  FTAG  \
0           0   5   8   0  13    4   1       Leicester 2018-08-10     1   
1           1   4   9   0  10    1   1         Cardiff 2018-08-11     0   
2           2   5  11   0  10    9   2  Crystal Palace 2018-08-11     2   
3           3   5   8   0  13    4   1         Chelsea 2018-08-11     3   
4           4   5  12   0  15    5   2       Tottenham 2018-08-11     2   

    ...     Home_Shots_Target Away_Shots_Target  Home_Corners  Away_Corners  \
0   ...                     0                 0             0             0   
1   ...                     0                 0             0             0   
2   ...                     0                 0             0             0   
3   ...                     0                 0             0             0   
4   ...                     0                 0             0             0   

   Home_Fouls  Away_Fouls  Home_Yellow  Away_Yellow  Home_Red Away_Red  
0

/mnt/c/Users/adams/OneDrive - University of Waterloo/Projects/Premier_League_Match_Predictor/DataCleaning.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataframe_temp=dataframe[(dataframe['HomeTeam']==team_name) | (dataframe['AwayTeam']==team_name)][dataframe['Date']<date]
/mnt/c/Users/adams/OneDrive - University of Waterloo/Projects/Premier_League_Match_Predictor/DataCleaning.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataframe_temp=dataframe[(dataframe['HomeTeam']==team_name) | (dataframe['AwayTeam']==team_name)][dataframe['Date']<date]


In [16]:
json_file=open('model.json','r')
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")
print("Model Has Been Loaded")

Model Has Been Loaded


In [24]:
x_test_real=data2[['Home_Points', 'Away_Points', 'Home_GD', 'Away_GD','G1.H.D',
       'G1.H.L', 'G1.H.W', 'G1.A.D', 'G1.A.L', 'G1.A.W', 'G2.H.D', 'G2.H.L',
       'G2.H.W', 'G2.A.D', 'G2.A.L', 'G2.A.W', 'G3.H.D', 'G3.H.L', 'G3.H.W',
       'G3.A.D', 'G3.A.L', 'G3.A.W', 'Home_Shots', 'Away_Shots','Home_Red', 'Away_Red','This_Year_LP_Home',
       'Last_Year_LP_Home', 'This_Year_LP_Away', 'Last_Year_LP_Away']]
y_test_real=data2['FTR.numerical']

In [30]:
predictions_real=loaded_model.predict(x_test_real)

In [31]:
def result_mapper(result):
    if result=='H':
        return 2
    elif result=='A':
        return 1
    else:
        return 0

In [33]:
def reverse_map(result):
    if result==2:
        return 'H'
    elif result==1:
        return 'A'
    else:
        return 'D'

In [34]:
predict_new=[]
for i in predictions_real:
    predict_new.append(reverse_map(np.argmax(i)))
real_new=[]     
for i in y_test_real:
    real_new.append(reverse_map(i))

In [36]:
from sklearn.metrics import classification_report,confusion_matrix

In [38]:
print(classification_report(predict_new,real_new))

             precision    recall  f1-score   support

          A       0.18      0.60      0.28        30
          D       0.36      0.18      0.24       118
          H       0.70      0.65      0.68       156

avg / total       0.52      0.46      0.47       304

